In [72]:
import pandas as pd
import numpy as np
import os
import json
import altair as alt
import numpy as np

In [106]:
#JSON_FILE_BUSTEDS_MH = "../analysis/13-datasets/BUSTEDS-MH/yokoyama.rh1.cds.mod.1-990.nex.BUSTEDS-MH.json"
#JSON_FILE_BUSTEDS = "../analysis/13-datasets/BUSTEDS/yokoyama.rh1.cds.mod.1-990.nex.BUSTEDS.json"
#JSON_FILE_MH = "../data/13-datasets/yokoyama.rh1.cds.mod.1-990.nex.FITTER.json"

JSON_FILE_BUSTEDS_MH = "../analysis/13-datasets/BUSTEDS-MH/HepatitisD.nex.BUSTEDS-MH.json"
JSON_FILE_BUSTEDS = "../analysis/13-datasets/BUSTEDS/HepatitisD.nex.BUSTEDS.json"
JSON_FILE_MH = "../data/13-datasets/HepatitisD.nex.FITTER.json"

pvalueThreshold_MH = 0.05
pvalueThreshold_BUSTEDS_MH = 0.1
ER_THRESHOLD = 5

In [107]:
def read_json(filename):
    print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

## Read the data

In [108]:
x = read_json(JSON_FILE_BUSTEDS_MH)
y = read_json(JSON_FILE_BUSTEDS)
z = read_json(JSON_FILE_MH)

# Reading: ../analysis/13-datasets/BUSTEDS-MH/HepatitisD.nex.BUSTEDS-MH.json
# Reading: ../analysis/13-datasets/BUSTEDS/HepatitisD.nex.BUSTEDS.json
# Reading: ../data/13-datasets/HepatitisD.nex.FITTER.json


In [109]:
z.keys()

dict_keys(['Evidence Ratios', 'Site Log Likelihood', 'Site substitutions', 'analysis', 'branch attributes', 'data partitions', 'fits', 'input', 'test results', 'tested', 'timers'])

In [110]:
x.keys()

dict_keys(['Evidence Ratios', 'Site Log Likelihood', 'Synonymous site-posteriors', 'analysis', 'background', 'branch attributes', 'data partitions', 'fits', 'input', 'test results', 'tested', 'timers'])

In [112]:
y.keys()

dict_keys(['Evidence Ratios', 'Site Log Likelihood', 'Synonymous site-posteriors', 'analysis', 'background', 'branch attributes', 'data partitions', 'fits', 'input', 'test results', 'tested', 'timers'])

In [113]:
x["Evidence Ratios"].keys()

dict_keys(['constrained', 'optimized null'])

In [114]:
#x["Evidence Ratios"]["constrained"][0]

## Assemble the data

In [115]:
## Assemble the data, pure ER's

In [116]:
data = x
num_sites = len(data["Evidence Ratios"]["constrained"][0])
res = [x+1 for x in range(num_sites)]
data_tuples = list(zip(res, data["Evidence Ratios"]["constrained"][0]))

# BUSTEDS-MH
df = pd.DataFrame(data_tuples, columns=['Site','BUSTEDS_MH_ER'])

for item in z["Evidence Ratios"].keys():
    df[item] = z["Evidence Ratios"][item][0]

# Finalize for plotting
#df["2ln(ER)"] = 2 * np.log(df["BUSTEDS_MH_ER"])
df

,Site,BUSTEDS_MH_ER,Double-hit vs Single-hit,Triple-hit islands vs Double-hit,Triple-hit vs Double-hit,Triple-hit vs Triple-hit islands
0,1,0.871342,1.004456,1.000139,0.999560,0.999421
1,2,0.891529,1.073489,1.000077,1.000340,1.000263
2,3,1.082308,0.982829,0.999634,0.976054,0.976411
3,4,2.209731,0.594572,1.001196,1.086627,1.085328
4,5,0.902786,1.071573,1.001481,0.922792,0.921427
...,...,...,...,...,...,...
191,192,1.274035,0.334232,0.998403,0.876706,0.878108
192,193,1.181448,0.839313,0.999512,1.017892,1.018389
193,194,0.984006,1.468912,0.999488,1.137587,1.138170
194,195,0.626256,0.801031,1.000940,0.995088,0.994153


## Transform the data, easier to plot

In [117]:
num_sites = len(x["Evidence Ratios"]["constrained"][0])
res = [x+1 for x in range(num_sites)]

# BUSTEDS-MH
bs = ["BUSTEDS-MH"] * num_sites
data_tuples = list(zip(res, x["Evidence Ratios"]["constrained"][0], bs))
# Site, ER_Value, Model
df1 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])

# BUSTEDS
bs = ["BUSTEDS"] * num_sites
data_tuples = list(zip(res, y["Evidence Ratios"]["constrained"][0], bs))
# Site, ER_Value, Model
df3 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])

# MH
#df = pd.DataFrame(data_tuples, columns=['Site','BUSTEDS_MH_ER'])
df_list = [] 
df_list.append(df1)
df_list.append(df3)

for item in z["Evidence Ratios"].keys():
    bs = [item] * num_sites
    data_tuples = list(zip(res, z["Evidence Ratios"][item][0], bs))
    df2 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])
    df_list.append(df2)
    

    
result = pd.concat(df_list)
result
# Finalize for plotting
#df["2ln(ER)"] = 2 * np.log(df["BUSTEDS_MH_ER"])
#df

,Site,Evidence Ratio,Method
0,1,0.871342,BUSTEDS-MH
1,2,0.891529,BUSTEDS-MH
2,3,1.082308,BUSTEDS-MH
3,4,2.209731,BUSTEDS-MH
4,5,0.902786,BUSTEDS-MH
...,...,...,...
191,192,0.878108,Triple-hit vs Triple-hit islands
192,193,1.018389,Triple-hit vs Triple-hit islands
193,194,1.138170,Triple-hit vs Triple-hit islands
194,195,0.994153,Triple-hit vs Triple-hit islands


## 2*ln(ER) data

In [118]:
data = x
num_sites = len(data["Evidence Ratios"]["constrained"][0])
res = [x+1 for x in range(num_sites)]
bs = ["BUSTEDS-MH"] * num_sites
data_tuples = list(zip(res, 2*np.log(data["Evidence Ratios"]["constrained"][0]), bs))

# Site, ER_Value, Model
df1 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])

# BUSTEDS-MH
#df = pd.DataFrame(data_tuples, columns=['Site','BUSTEDS_MH_ER'])
df_list = [] 
df_list.append(df1)
for item in z["Evidence Ratios"].keys():
    bs = [item] * num_sites
    data_tuples = list(zip(res, 2*np.log(z["Evidence Ratios"][item][0]), bs))
    df2 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])
    df_list.append(df2)
    

    
result_logn = pd.concat(df_list)
result_logn
# Finalize for plotting
#df["2ln(ER)"] = 2 * np.log(df["BUSTEDS_MH_ER"])
#df

,Site,Evidence Ratio,Method
0,1,-0.275442,BUSTEDS-MH
1,2,-0.229634,BUSTEDS-MH
2,3,0.158191,BUSTEDS-MH
3,4,1.585742,BUSTEDS-MH
4,5,-0.204539,BUSTEDS-MH
...,...,...,...
191,192,-0.259971,Triple-hit vs Triple-hit islands
192,193,0.036444,Triple-hit vs Triple-hit islands
193,194,0.258844,Triple-hit vs Triple-hit islands
194,195,-0.011729,Triple-hit vs Triple-hit islands


## Pure ER's between MH, BUSTEDS, and BUSTEDS-MH

In [119]:
print(result[result["Site"] == 13])

    Site  Evidence Ratio                            Method
12    13        6.055258                        BUSTEDS-MH
12    13       25.708904                           BUSTEDS
12    13        5.787431          Double-hit vs Single-hit
12    13        0.995481  Triple-hit islands vs Double-hit
12    13        1.714273          Triple-hit vs Double-hit
12    13        1.722055  Triple-hit vs Triple-hit islands


In [96]:
print(result[result["Site"] == 277])

     Site  Evidence Ratio                            Method
276   277       17.713717                        BUSTEDS-MH
276   277      318.508618                           BUSTEDS
276   277      122.223212          Double-hit vs Single-hit
276   277        0.826448  Triple-hit islands vs Double-hit
276   277        1.995188          Triple-hit vs Double-hit
276   277        2.414171  Triple-hit vs Triple-hit islands


## Visualize

In [131]:
source = result

alt.Chart(source).mark_area().encode(
    x='Site',
    y='Evidence Ratio',
    color='Method',
    stroke='Method',
).properties(
    width=1200,
    height=800)


alt.Chart(...)

In [170]:
r1 = result[result["Method"] == "BUSTEDS-MH"]
r2 = result[result["Method"] == "BUSTEDS"]

source = pd.concat([r1,r2])

source = source.fillna(0)


source["Evidence Ratio"] = pd.to_numeric(source["Evidence Ratio"], downcast="float")
source["Site"] = pd.to_numeric(source["Site"], downcast="float")

line = alt.Chart(source).mark_area().encode(
    x='Site',
    y='sum(Evidence Ratio)',
    color='Method'
).properties(
    width=1200,
    height=800)

line


alt.Chart(...)

In [121]:
source = result_logn

alt.Chart(source).mark_line(opacity=0.8).encode(
    x='Site',
    y=alt.Y('Evidence Ratio', axis=alt.Axis(title='2*ln(Evidence Ratio)')),
    color='Method',
    stroke='Method',
).properties(
    width=1200,
    height=800)


alt.Chart(...)

In [52]:
source = df

line = alt.Chart(source).mark_line().encode(
    x='Site',
    y='BUSTEDS_MH_ER', 
).properties(
    width=800,
    height=600)

line1 =alt.Chart(source).mark_line(color="red").encode(
    x='Site',
    y='Double-hit vs Single-hit', 
).interactive()

# Plot
line + line1

alt.LayerChart(...)